# Load in dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
data = pd.read_csv("Datasets.csv")

In [16]:
parameters = list(data.columns)
cell_data = data.filter(items=parameters[:-1])

# Image generation

In [33]:
from math import sin, cos, radians

In [151]:
"""
Takes in a Pandas series and generates an image of the re-entrant auxetic
    described by the parameters

Arguments:
* data: Pandas series with 'Slant cell length', 'Cell thickness', 'Cell angle', 
    'Vertical cell length'
* show: False if plot should not be displayed
* save: True if the plot should be saved
* fname: name of the file to save the plot into if save=True
"""
def gen_image(data, show=True, save=False, fname=''):
    # ignoring cell thickness for now
    slant_len = data["Slant cell length"]
    thickness = data["Cell thickness"]
    angle = data["Cell angle"]
    height = data["Vertical cell length"]

    # for calculating x and y coordinates of plot lines
    slant_dx = slant_len * cos(radians(90 - angle))
    slant_dy = slant_len * sin(radians(90 - angle))

    # plot point generation
    top = 1 + height
    bottom = 1
    center = 1 + slant_dx
    left = 1
    right = 1 + 2*slant_dx
    # [ [[x values of 1st slant], [y values of 1st slant]], [...], [...], [...] ]
    slants = [
        [[left, center], [bottom, bottom + slant_dy]],
        [[center, right], [bottom + slant_dy, bottom]],
        [[left, center], [top, top - slant_dy]],
        [[center, right],[top - slant_dy, top]]
    ]
    # vertical lines
    verticals = [
        [[left, left], [bottom, top]],
        [[center, center], [bottom + slant_dy, bottom - 0.2*height]],
        [[right, right], [bottom, top]],
        [[center, center], [top - slant_dy, top + 0.2*height]]
    ]

    # plotting and saving
    plt.cla()
    for slant in slants:
        plt.plot(*slant, color='black', linewidth=thickness*0.5)
    for line in verticals:
        plt.plot(*line, color='black', linewidth=thickness*0.5)
    plt.axis('off')
    
    if save:
        plt.savefig(fname)
    if not show:
        plt.close()

"""
Saves a group of images

Arguments:
* data: Pandas dataframe
* indices: list of indices of rows in the dataframe to
    generate images from
"""
def save_images(data, indices, folder='test'):
    excluded = []
    for idx in indices:
        if idx >= data.shape[0]:
            excluded.append(idx)
            continue
        gen_image(data.iloc[idx], show=False, save=True, fname=f'./images/{folder}/{idx}.png')
    if excluded:
        print(f"indices excluded (not in range): {excluded}")

## Train vs. Test Split

In [117]:
import random

In [ ]:
train = data.sample(n=1500)
test = data.sample(n=2500)
print(f'{train.shape[0]} training samples')
print(f'{test.shape[0]} test samples')

Checking distribution of each parameter

In [ ]:
# NOT NECESSARY FOR MODEL
# original data
for param in parameters[:-1]:
    values = list(data[param])
    unique_vals = set(values)
    plt.title(f"Original: {param}")
    plt.hist(values, bins=len(unique_vals), edgecolor='black')
    plt.show()

In [ ]:
# NOT NECESSARY FOR MODEL
# train data
for param in parameters[:-1]:
    values = list(train[param])
    unique_vals = set(values)
    plt.title(f"Train: {param}")
    plt.hist(values, bins=len(unique_vals), edgecolor='black')
    plt.show()

In [ ]:
# NOT NECESSARY FOR MODEL
# test data
for param in parameters[:-1]:
    values = list(test[param])
    unique_vals = set(values)
    plt.title(f"Test: {param}")
    plt.hist(values, bins=len(unique_vals), edgecolor='black')
    plt.show()

Generate ground truth labels and save images

In [ ]:
train_indices = list(train.index)
test_indices = list(test.indices)

In [175]:
# Train labels
train_labels = []
for idx in train_indices:
    poisson_ratio = data.iloc[idx]["Poisson's ratio"]
    train_labels.append(poisson_ratio)

In [ ]:
# Test labels
test_labels = []
for idx in test_indices:
    poisson_ratio = data.iloc[idx]["Poisson's ratio"]
    test_labels.append(poisson_ratio)

In [ ]:
# Save training images
save_images(data, train_indices, folder='train')

In [ ]:
# Save test images
save_images(data, test_indices, folder='test')

# Model training